In [135]:
import pandas as pd
import numpy as np

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.decomposition import IncrementalPCA

In [180]:
df = pd.read_csv('Desktop/Homework/dataset.csv', index_col=0)
df_test = pd.read_csv('Desktop/Homework/dataset_pruebas.csv',index_col=0)

In [137]:
df.shape

(428696, 11)

# Hago diferentes particiones del dataset, busco la que me de una mejor estimación

In [138]:
df_a = df.dropna()

Superficies con 0 metros cuadrados?

In [139]:
df_a.groupby('surface_covered_in_m2').count()

,lat,lon,price_aprox_usd,rooms,state_n,barrio_n,property_type_n,pileta,jardin,garage
surface_covered_in_m2,,,,,,,,,,
0.0,3577,3577,3577,3577,3577,3577,3577,3577,3577,3577
1.0,211,211,211,211,211,211,211,211,211,211
2.0,21,21,21,21,21,21,21,21,21,21
3.0,34,34,34,34,34,34,34,34,34,34
4.0,37,37,37,37,37,37,37,37,37,37
5.0,13,13,13,13,13,13,13,13,13,13
6.0,27,27,27,27,27,27,27,27,27,27
7.0,16,16,16,16,16,16,16,16,16,16
8.0,7,7,7,7,7,7,7,7,7,7


In [140]:
df_a = df_a.loc[df_a['surface_covered_in_m2']>=10.0, :]

In [144]:
df_d = df_a.sample(frac=1)
prices_d = df_d['price_aprox_usd']
del df_d['price_aprox_usd']

In [145]:
price_a = df_a['price_aprox_usd']
del df_a['price_aprox_usd']

In [146]:
df_a.shape

(195876, 10)

In [147]:
random_forest = RandomForestRegressor()
random_forest.get_params()

{'bootstrap': True,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': 1,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [148]:
df_d.head()

,lat,lon,rooms,surface_covered_in_m2,state_n,barrio_n,property_type_n,pileta,jardin,garage
585456,-34.812611,-58.496278,3.0,68.0,3,74,2,0,0,0
215018,-34.569235,-58.489231,4.0,130.0,1,44,1,0,0,0
774736,-34.617804,-58.576052,3.0,56.0,4,83,1,0,0,0
85791,-34.604309,-58.513123,3.0,60.0,1,18,1,0,0,0
189702,-34.631513,-58.392849,1.0,38.0,1,25,1,0,0,0


In [149]:
param_grid= {'max_features': range(1, 11)}
grid_drop = GridSearchCV(random_forest, param_grid, cv=10)
grid_drop.fit(df_d, prices_d)
print(grid_drop.best_params_, grid_drop.best_score_)

({'max_features': 2}, 0.96413579382011882)


#  Preprocesamiento con QuantileTransformer y RobustScaler (Inutiles en decision tree en gral, como RandomForest)

In [210]:
q = QuantileTransformer()
X_train = q.fit_transform(df_d, prices_d)
#X_train = pd.DataFrame(data=X_train[0:,0:])
param_grid= {'max_features': range(1, 11)}
grid_q = GridSearchCV(random_forest, param_grid, cv=10)
grid_q.fit(X_train, prices_d)
print(grid_q.best_params_, grid_q.best_score_)

({'max_features': 3}, 0.96449214910616154)


In [211]:
r = RobustScaler()
X_train = r.fit_transform(df_d, prices_d)
#X_train = pd.DataFrame(data=X_train[0:,0:])
param_grid= {'max_features': range(1, 11)}
grid_r = GridSearchCV(random_forest, param_grid, cv=10)
grid_r.fit(X_train, prices_d)
print(grid_r.best_params_, grid_r.best_score_)

({'max_features': 1}, 0.96458502355803422)


# Testeo

In [181]:
df_test.shape

(228867, 11)

Armo un set de testeo de aproximadamente 10.000 datos

In [183]:
test = df_test.sample(frac=0.06)
prices_test = test['price_aprox_usd']
del test['price_aprox_usd']

In [184]:
test.shape

(13732, 10)

In [185]:
def superficie_valor(row):
    if not(np.isnan(row['surface_covered_in_m2'])):
        if row['surface_covered_in_m2']>=10.0:
            return row['surface_covered_in_m2']
        else:
            return df_a.loc[:, 
                   ['property_type_n', 'surface_covered_in_m2']].groupby('property_type_n').median().iloc[int(row['property_type_n'])-1, 0]            
    #return df_a.loc[df_d['surface_covered_in_m2']<=200.0, 
    #                ['property_type_n', 'surface_covered_in_m2']].groupby('property_type_n').mean().iloc[int(row['property_type_n'])-1, 0]
    return df_a.loc[:, 
           ['property_type_n', 'surface_covered_in_m2']].groupby('property_type_n').median().iloc[int(row['property_type_n'])-1, 0]

test['surface_covered_in_m2'] = test.apply(superficie_valor, axis=1)

In [186]:
def lat_valor(row):
    if not(np.isnan(row['lat'])):
        return row['lat']
    
    return df_a.loc[:, 
            ['barrio_n', 'lat']].groupby('barrio_n').median().iloc[int(row['barrio_n'])-1, 0]

test['lat'] = test.apply(lat_valor, axis=1)

In [187]:
def lon_valor(row):
    if not(np.isnan(row['lon'])):
        return row['lon']
    
    return df_a.loc[:, 
            ['barrio_n', 'lon']].groupby('barrio_n').median().iloc[int(row['barrio_n'])-1, 0]

test['lon'] = test.apply(lon_valor, axis=1)

In [188]:
values = { 
          'rooms': np.trunc(df_a['rooms'].median()),
         }

test = test.fillna(value=values)

In [189]:
test.head()

,lat,lon,rooms,surface_covered_in_m2,state_n,barrio_n,property_type_n,pileta,jardin,garage
630593,-34.918684,-57.959919,3.0,285.0,3,72,2,0,0,0
233166,-34.596732,-58.441032,3.0,250.0,1,35,3,0,0,0
102138,-34.615747,-58.371739,3.0,120.0,1,23,3,0,0,0
537537,-34.479726,-58.522148,3.0,344.0,2,69,2,0,0,0
737725,-34.667453,-58.560942,4.0,260.0,4,84,2,0,0,0


In [190]:
prediccion = grid_drop.best_estimator_.predict(test)

Un error de 12 cifras, muy parecido al de kaggle (estoy tomando un set de pruebas como el de kaggle)

In [191]:
mean_squared_error(prices_test, prediccion)

104464105923.93326

# Crear dataset para kaggle

In [192]:
properati = pd.read_csv('Desktop/Homework/properati.csv')

In [193]:
properati.index = properati['id']
del properati['id']

In [194]:
properati.head()

,lat,lon,rooms,surface_covered_in_m2,state_n,barrio_n,property_type_n,pileta,jardin,garage
id,,,,,,,,,,
3632,-34.610988,-58.363464,NaN,NaN,1,22,1,0,0,1
3633,NaN,NaN,NaN,NaN,5,4,1,1,0,0
2263404,-34.589363,-58.412880,NaN,48.0,1,12,1,0,0,0
2263405,NaN,NaN,NaN,39.0,1,45,1,0,0,0
2263406,NaN,NaN,NaN,51.0,1,45,1,0,0,0


In [198]:
properati['surface_covered_in_m2'] = properati.apply(superficie_valor, axis=1)

In [199]:
properati['lat'] = properati.apply(lat_valor, axis=1)

In [200]:
properati['lon'] = properati.apply(lon_valor, axis=1)

In [201]:
values = { 
          'rooms': np.trunc(df_a['rooms'].median()),
         }

properati = properati.fillna(value=values)

In [202]:
properati.head()

,lat,lon,rooms,surface_covered_in_m2,state_n,barrio_n,property_type_n,pileta,jardin,garage
id,,,,,,,,,,
3632,-34.610988,-58.363464,3.0,57.0,1,22,1,0,0,1
3633,-34.595814,-58.478161,3.0,57.0,5,4,1,1,0,0
2263404,-34.589363,-58.412880,3.0,48.0,1,12,1,0,0,0
2263405,-34.586400,-58.450287,3.0,39.0,1,45,1,0,0,0
2263406,-34.586400,-58.450287,3.0,51.0,1,45,1,0,0,0


In [203]:
prediccion_properati = grid_drop.best_estimator_.predict(properati)

In [204]:
predicciones = pd.DataFrame({'price_usd':prediccion_properati})
predicciones.index = properati.index

In [208]:
predicciones.head()

,price_usd
id,
3632,513200.000000
3633,123165.000000
2263404,171000.000000
2263405,117915.882353
2263406,120770.000000


In [209]:
predicciones.to_csv('Desktop/Homework/prediction.csv')